In [1]:
#include <iostream>
#include <algorithm>

# Search Trees

## Abstract Sorted Lists

- In the **Abstract List ADT**, the objects are explicitly linearly ordered by the programmer

- Within the **Abstract Sorted List ADT** the relation is based on an **implicit** linear ordering
    - Certain operations no longer make sense
        `push_front` and `push_back` are replaced by a generic insert
        
- Queries that may be made about data stored in a **Sorted List ADT** include
    - Finding the smallest and largest values
    - Finding the $k$th largest value
    - Find the next larger and previous smaller objects of a given object which may or may not be in the container
    - Iterate through those objects that fall on an interval $[a, b]$

## Implementation

- If we implement an **Abstract Sorted List** using an array or a linked list, we will have operations which are $O(n)$
    - As an insertion could occur anywhere in a linked list or array, we must either traverse or copy, on average, $O(n)$ objects

## Binary Search Trees

- A binary tree dictates an order on the two children

- Exploit this order to	improve performance
    - Require all objects in the left sub-tree to be less than the object stored in the root node, and
    - Require all objects in the right sub-tree to be greater than the object in the root object
    
![](../img/bs-tree1.png)

- Each of the two sub-trees will themselves be binary search trees

- We can already use this structure for searching:
    - examine the root node and if we have not found what we are looking for
        - If the object is less than what is stored in the root node, continue searching in the left sub-tree
        - Otherwise, continue searching the right sub-tree

- With a linear order, one of the following three must be true
    - `a < b`
    - `a = b`
    - `a > b`

## Definition

- A non-empty **binary search tree** as a binary tree with the following properties
    - The left sub-tree (if any) is a binary search tree and all values are less than the root value, and
    - The right sub-tree (if any) is a binary search tree and all values are greater than the root value
    
![](../img/bs-tree2.png)

- Unfortunately, it is possible to construct degenerate binary search trees
    - This is equivalent to a linked list, i.e., $O(n)$

![](../img/bs-tree3.png)

- All these binary search trees store the same data

## Duplicate values

- Assume that in any binary tree, we are **not storing** duplicate values unless otherwise stated
    - In reality, it is seldom the case where duplicate values in a container must be stored as separate entities

- You can always consider duplicate values with modifications to the algorithms we will cover

## Implementation

- An implementation of a binary search tree follows the blueprint of `SinglyLinkedList` class
    - we will have a `BinarySearchNode` class
    - a `BinarySearchTree` class will store a pointer to the root

- We will use templates, however, we will require that the class overrides the comparison operators
- Any class which uses this binary search tree class must therefore implement comparison operators for the stored values, e.g.
```cpp
bool operator<( Type const &, Type const & );
```

In [2]:
#include "../src/BinaryNode.h"

template <typename Type>
class BinarySearchTree;

template <typename Type>
class BinarySearchNode : public BinaryNode<Type> {
    using BinaryNode<Type>::element;
    using BinaryNode<Type>::left_tree;
    using BinaryNode<Type>::right_tree;

public:
    BinarySearchNode( const Type& );

    // Accessors
    BinarySearchNode* left() const;
    BinarySearchNode* right() const;
    Type front() const; // minimum object
    Type back() const;  // maximum object
    bool find( const Type & ) const;
    Type next( Type const & ) const;
    Type at( int k ) const;
    void print( int = 0 );

    // Mutators
    bool insert( const Type &, BinarySearchNode *& = nullptr );
    bool erase( const Type &, BinarySearchNode *& = nullptr );

    // Friends
    friend class BinarySearchTree<Type>;
    
    template <typename T>
    friend std::ostream &operator<<( std::ostream &, const BinarySearchNode<T> & );
};

### Constructor

- The constructor simply calls the constructor of the base class
    - Recall that it sets both `left_tree` and `right_tree` to `nullptr`
    - It assumes that this is a new leaf node

In [3]:
template <typename Type>
BinarySearchNode<Type>::BinarySearchNode( const Type &e ) :
    BinaryNode<Type>( e ){
    // Just calls the constructor of the base class
}

### Accessors

- Because it is a derived class, it already inherits the function:
```cpp
Type value() const
```
- Because the base class returns a pointer to a `BinaryNode`, we must recast them as `BinarySearchNode`

In [4]:
template <typename Type>
BinarySearchNode<Type> *BinarySearchNode<Type>::left() const {
    return reinterpret_cast<BinarySearchNode *>( BinaryNode<Type>::left() );
}

template <typename Type>
BinarySearchNode<Type> *BinarySearchNode<Type>::right() const {
    return reinterpret_cast<BinarySearchNode *>( BinaryNode<Type>::right() );
}

- Following member functions are inherited from the base class `BinaryNode`
```cpp
bool is_leaf() const
int size() const
int height() const
```

### Finding the Minimum Object

- To find the node containing the smallest elements in the tree (its front)
    - start at the root and go left as long as there is a left child.
    - the stopping point is the smallest element
- The run time $O(h)$
![](../img/bs-tree-min.png)

In [5]:
template <typename Type>
Type BinarySearchNode<Type>::front() const {
    return ( left() == nullptr ) ? this->value() : left()->front();
}

### Finding the Maximum Object

- To find the node containing the largest elements in the tree (its back)
    - start at the root and go right as long as there is a right child.
    - the stopping point is the largest element.
- The run time $O(h)$
![](../img/bs-tree-max.png)

In [6]:
template <typename Type>
Type BinarySearchNode<Type>::back() const {
    return ( right() == nullptr ) ? this->value() : right()->back();
}

### Find

- To determine membership, traverse the tree based on the linear relationship
    - If a node containing the value is found, e.g., `81`, return `true`
    ![](../img/bs-tree-find1.png)
    - If an empty node is reached, e.g., `36`, the object is not in the tree:
    ![](../img/bs-tree-find2.png)

- The implementation is similar to `front` and `back`
    - The run time is $O(h)$

In [7]:
template <typename Type>
bool BinarySearchNode<Type>::find( const Type &obj ) const {
    if ( this->value() == obj ) {
        return true;
    } else if ( this->value() < obj && right() != nullptr) {
        return right()->find( obj );
    } else if ( this->value() > obj && left() != nullptr) {
        return left()->find( obj );
    }
    return false;
}

## Insert

- Recall that a **Sorted List** is implicitly ordered
    - It does not make sense to have member functions such as `push_front` and `push_back`
    - Insertion will be performed by a single insert member function which places the object into the correct location

- An insertion will be performed at a leaf node
    - Any empty node is a possible location for an insertion
    ![](../img/bs-tree-insert1.png)
- The values which may be inserted at any empty node depend on the surrounding nodes

- For example, this node may hold `48`, `49`, or `50`
![](../img/bs-tree-insert2.png)

- An insertion at this location must be `35`, `36`, `37`, or `38`
![](../img/bs-tree-insert3.png)

- This empty node may hold values from `71` to `74`
![](../img/bs-tree-insert4.png)

- Like `find`, we will step through the tree
    - If we find the object already in the tree, stop
        - The object is already in the binary search tree (no duplicates)
    - Otherwise, we will arrive at an empty node
    - The object will be inserted into that location
- The run time is $O(h)$

- In inserting the value `52`, we traverse the tree until we reach an empty node
    - The left sub-tree of `54` is an empty node
    ![](../img/bs-tree-insert5.png)

- A new leaf node is created and assigned to the member variable `left_tree`
![](../img/bs-tree-insert6.png)

- In inserting `40`, we determine the right sub-tree of `39` is an empty node
![](../img/bs-tree-insert7.png)

- A new leaf node storing `40` is created and assigned to the member variable `right_tree`
![](../img/bs-tree-insert8.png)

In [8]:
template <typename Type>
bool BinarySearchNode<Type>::insert( const Type &obj, BinarySearchNode* &t ) {
    if ( t == nullptr ) {
        t = new BinarySearchNode<Type>( obj );
        return true;
    } else if ( obj < t->value() ) {
        return insert( obj, reinterpret_cast<BinarySearchNode<Type>*&>(t->left_tree) );
    } else if ( obj > t->value() ) {
        return insert( obj, reinterpret_cast<BinarySearchNode<Type>*&>(t->right_tree) );
    } else {
        return false;
    }
}

It is assumed that if neither of the conditions
- `obj < value()`
- `obj > value()`

then `obj == value()` and therefore we do nothing
- The object is already in the binary search tree	

In [9]:
template <typename Type>
void BinarySearchNode<Type>::print( int depth ) {
    auto print_tabs = [](int depth) { std::string tabs(depth, '\t'); std::cout << tabs; };
    print_tabs( depth );
    std::cout << this->value() << std::endl;
    if (left() != nullptr)
        left()->print(depth+1);
    if (right() != nullptr)
        right()->print(depth+1);
}

In [10]:
{
    BinarySearchNode<int>* r = new BinarySearchNode<int>(42);
    for(int i : {39,11,8,3,29,19,16,24,34})
        r->insert(i, r);
    std::cout << "Before `40` added" <<std::endl;    
    r->print();    
    r->insert(40, r);
    std::cout << "After `40` added" <<std::endl;    
    r->print();
    r->clear();
}

Before `40` added
42
	39
		11
			8
				3
			29
				19
					16
					24
				34
After `40` added
42
	39
		11
			8
				3
			29
				19
					16
					24
				34
		40


### Exercise

- In the given order, insert these objects into an initially empty binary search tree:
```
    31  45  36  14  52  42  6  21  73  47  26 37  33  8
``` 
- What values could be placed:
    - To the left of 21?
    - To the right of 26?
    - To the left of 47?
- How would we determine if 40 is in this binary search tree?
- Which values could be inserted to increase the height of the tree?

In [11]:
{
    BinarySearchNode<int>* r = new BinarySearchNode<int>(31);
    for(int i : {45,36,14,52,42,6,21,73,47,26,37,33,8})
        r->insert(i, r);
    std::cout << "Height: " << r->height() << std::endl;
    r->print();
    r->insert(40, r);
    std::cout << "Height: " << r->height() << std::endl;
    r->print();
    r->clear();
}

Height: 5
31
	14
		6
			8
		21
			26
	45
		36
			33
			42
				37
		52
			47
			73
Height: 6
31
	14
		6
			8
		21
			26
	45
		36
			33
			42
				37
					40
		52
			47
			73


## Erase

- A node being erased is not always going to be a leaf node
- There are three possible scenarios:
    - The node is a leaf node,
    - It has exactly one child, or
    - It has two children (it is a full node)

- A leaf node simply must be removed and the appropriate member variable of the parent is set to `nullptr`
    - Consider removing `75`
![](../img/bs-tree-erase1.png)

- The node is deleted and `left_tree` of `81` is set to `nullptr`
![](../img/bs-tree-erase2.png)

- Erasing the node containing `40` is similar
![](../img/bs-tree-erase3.png)

- The node is deleted and `right_tree` of `39` is set to `nullptr`
![](../img/bs-tree-erase4.png)

- If a node has only one child, we can simply promote the sub-tree associated with the child
    - Consider removing 8 which has one left child
![](../img/bs-tree-erase5.png)

- The node `8` is deleted and the `left_tree` of `11` is updated to point to `3`
![](../img/bs-tree-erase6.png)

- There is no difference in promoting a single node or a sub-tree
    - To remove `39`, it has a single child `11`
![](../img/bs-tree-erase7.png)

- The node containing `39` is deleted and `left_node` of `42` is updated to point to `11`
    - Notice that order is still maintained
![](../img/bs-tree-erase8.png)

- Consider erasing the node containing `99`
![](../img/bs-tree-erase9.png)

- 	The node is deleted and the left sub-tree is promoted
    - The member variable right_tree of `70` is set to point to `92`
    - Again, the order of the tree is maintained
![](../img/bs-tree-erase10.png)

- Finally, we will consider the problem of erasing a full node, e.g., `42`
- We will perform two operations
    - Replace `42` with the minimum object in the right sub-tree
    - Erase that object from the right sub-tree

![](../img/bs-tree-erase11.png)

- In this case, we replace `42` with `47`
    - We temporarily have two copies of `47` in the tree

![](../img/bs-tree-erase12.png)

- We now recursively erase `47` from the right sub-tree
    - We note that `47` is a leaf node in the right sub-tree

![](../img/bs-tree-erase13.png)

- Leaf nodes are simply removed and `left_tree` of `51` is set to `nullptr`
    - Notice that the tree is still sorted
		- `47` was the least object in the right sub-tree

![](../img/bs-tree-erase14.png)

- Suppose we want to erase the root `47` again
    - We must copy the minimum of the right sub-tree
    - We could promote the maximum object in the left sub-tree and achieve similar results

![](../img/bs-tree-erase15.png)

- We copy `51` from the right sub-tree
![](../img/bs-tree-erase16.png)

- We must proceed by delete `51` from the right sub-tree
![](../img/bs-tree-erase17.png)

- In this case, the node storing `51` has just a single child
![](../img/bs-tree-erase18.png)

- We delete the node containing `51` and assign the member variable `left_tree` of `70` to point to `59`
![](../img/bs-tree-erase19.png)
- Note that after seven removals, the remaining tree is still correctly sorted

- In the two examples of removing a full node, we promoted:
    - A node with no children
    - A node with right child
- Is it possible, in removing a full node, to promote a child with two children?

| |
|-|
|![](../img/bs-tree-erase12.png) |
|![](../img/bs-tree-erase16.png) |

- Recall that we promoted the minimum value in the right sub-tree
    - If that node had a left sub-tree, that sub-tree would contain a smaller value
    
| |
|-|
|![](../img/bs-tree-erase12.png) |
|![](../img/bs-tree-erase16.png) |

In [12]:
template <typename Type>
bool BinarySearchNode<Type>::erase( Type const &obj, BinarySearchNode *&t ) {
    if (  t == nullptr ) {
        return false;  // Item not found; do nothing
    } else if ( obj < t->value() ) {
        return erase( obj,  reinterpret_cast<BinarySearchNode<Type>*&>(t->left_tree) );
    } else if ( obj > t->value() ) {
        return erase( obj, reinterpret_cast<BinarySearchNode<Type>*&>(t->right_tree) );
    } else if ( t->left() != nullptr && t->right() != nullptr ) {   // full node
        t->element = t->right()->front();
        erase( t->value(), reinterpret_cast<BinarySearchNode<Type>*&>(t->right_tree) );
    } else { // only one child or leaf
        BinarySearchNode<Type> *oldNode = t;
        t = ( t->left() != nullptr ) ? t->left() : t->right();
        delete oldNode;
    }
    return true;
}

In [13]:
{
    BinarySearchNode<int>* r = new BinarySearchNode<int>(42);
    for(int i : {39,11,8,3,29,19,16,24,34,70,51,47,59,54,52,68,63,99,92,81,75,87,96})
        r->insert(i, r);
    std::cout << "Before `75` removed" <<std::endl;    
    r->print();
    r->erase(75, r);
    std::cout << "After `75` removed" <<std::endl;    
    r->print();
    r->clear();
}

Before `75` removed
42
	39
		11
			8
				3
			29
				19
					16
					24
				34
	70
		51
			47
			59
				54
					52
				68
					63
		99
			92
				81
					75
					87
				96
After `75` removed
42
	39
		11
			8
				3
			29
				19
					16
					24
				34
	70
		51
			47
			59
				54
					52
				68
					63
		99
			92
				81
					87
				96


### Exercise

- In the binary search tree generated previous exercise:
    - Erase 47
    - Erase 21
    - Erase 45
    - Erase 31
    - Erase 36

In [14]:
{
    BinarySearchNode<int>* r = new BinarySearchNode<int>(31);
    for(int i : {45,36,14,52,42,6,21,73,47,26,37,33,8})
        r->insert(i, r);
    std::cout << "Height: " << r->height() << std::endl;
    r->print();
    r->erase(47, r);
    std::cout << "Height: " << r->height() << std::endl;
    r->print();
    r->clear();
}

Height: 5
31
	14
		6
			8
		21
			26
	45
		36
			33
			42
				37
		52
			47
			73
Height: 5
31
	14
		6
			8
		21
			26
	45
		36
			33
			42
				37
		52
			73


## Binary Search Tree

- Now, we introduce a container which stores the root, `BinarySearchTree` class
- Most operations will be simply passed to the root node

In [15]:
template <typename Type>
class BinarySearchTree {
    BinarySearchNode<Type> *root_node;
public:
    BinarySearchTree();
    ~BinarySearchTree();

    BinarySearchNode<Type>* root() const;
    bool empty() const;
    int size() const;
    int height() const;
    Type front() const;
    Type back() const;
    bool find( const Type &obj ) const;
    Type at( int k ) const;

    void clear();
    bool insert( const Type &obj );
    bool erase( const Type &obj );
};

### Constructor & Destructor

In [16]:
template <typename Type>
BinarySearchTree<Type>::BinarySearchTree(): root_node( nullptr ) {
    // does nothing
}

In [17]:
template <typename Type>
BinarySearchTree<Type>::~BinarySearchTree() {
    clear();
}

In [18]:
template <typename Type>
void BinarySearchTree<Type>::clear() {
    root()->clear();
}

### Capacity

In [19]:
template <typename Type>
BinarySearchNode<Type>* BinarySearchTree<Type>::root() const {
    return root_node;
}

In [20]:
template <typename Type>
bool BinarySearchTree<Type>::empty() const {
    return root() == nullptr;
}

In [21]:
template <typename Type>
int BinarySearchTree<Type>::size() const {
    return empty() ? 0 : root()->size();
}

In [22]:
template <typename Type>
int BinarySearchTree<Type>::height() const {
    return empty() ? -1 : root()->height();
}

### Accessors

In [23]:
template <typename Type>
Type BinarySearchTree<Type>::front() const {
    return root()->front();
}

In [24]:
template <typename Type>
Type BinarySearchTree<Type>::back() const {
    return root()->back();
}

In [25]:
template <typename Type>
bool BinarySearchTree<Type>::find( Type const &obj ) const {
     return root()->find( obj );
}

### Modifiers

In [26]:
template <typename Type>
bool BinarySearchTree<Type>::insert( Type const &obj ) {
    if ( empty() ) {
        root_node = new BinarySearchNode<Type>(obj);
        return true;
    }
    return root()->insert( obj, root_node );
}

In [27]:
template <typename Type>
bool BinarySearchTree<Type>::erase( Type const &obj ) {
    return root()->erase( obj, root_node );
}

## Previous and Next Objects

- All the operations up to now have been operations which work on any container: `count`, `insert`, etc.

- Operations specific to *linearly ordered data* include
    - Find the next larger and previous smaller objects of a given object which may or may not be in the container
    - Find the $k$th value of the container
    - Iterate through those objects that fall on an interval $[a, b]$

- We will focus on finding the next largest object

- To find the next largest object
    - If the node has a right sub-tree, the minimum object in that sub-tree is the next-largest object 
    
![](../img/bs-tree-next1.png)

- If, however, there is no right sub-tree:
    - It is the next largest object (if any) that exists in the path from the root to the node
![](../img/bs-tree-next2.png)

- More generally:  what is the next largest value of an arbitrary object?
    - This can be found with a single search from the root node to one of the leaves - an $O(h)$ operation
- This function returns the object if it did not find something greater than it

In [28]:
template <typename Type>
Type BinarySearchNode<Type>::next( const Type &obj ) const {
    if ( this->is_leaf() ) {
        return obj;
    } else if ( this->value() == obj  ) {
        return ( right() == nullptr ) ? obj : right()->front();
    } else if ( this->value() > obj ) {
        Type tmp = left()->next( obj );
        return ( tmp == obj ) ? this->value() : tmp;
    } else {
        return right()->next( obj );
    }
}

In [29]:
{
    BinarySearchTree<int> t;
    for(int i : {42,39,11,8,3,29,19,16,24,34,70,51,47,59,54,52,68,63,99,92,81,75,87,96})
        t.insert(i);
    for(int i : {75,8,39,99,42,47})
        t.erase(i);
    auto r = t.root(); r->print();
    std::cout << "Next op 3: " << r->next(3) << std::endl;
    std::cout << "Next op 63: " << r->next(63) << std::endl;
    std::cout << "Next op 51: " << r->next(51) << std::endl;
    std::cout << "Next op 59: " << r->next(59) << std::endl;
}

51
	11
		3
		29
			19
				16
				24
			34
	70
		59
			54
				52
			68
				63
		92
			81
				87
			96
Next op 3: 11
Next op 63: 68
Next op 51: 52
Next op 59: 63


## Finding the $k$th Object

- Another operation on sorted lists may be finding the $k$th largest object
    - Recall that $k$ goes from $0$ to $n-1$, and $l$ is the size of the left sub-tree
    - If the left sub-tree has $l = k$ values, return the current node,
    - If the left sub-tree has $l > k$ values, return the $k$th value of the left sub-tree,
    - Otherwise, the left sub-tree has $l < k$ values, so return the $(k-l-1)$th value of the right sub-tree
![](../img/bs-tree-kth.png)

In [30]:
template <typename Type>
Type BinarySearchNode<Type>::at( int k ) const {
    int l = left() == nullptr ? 0 : left()->size();
    if ( l == k ) {
        return this->value();
    } else if ( l > k ) {
        return left()->at( k );
    } else {
        return right()->at( k - l - 1 );
    }
}

In [31]:
template <typename Type>
Type BinarySearchTree<Type>::at( int k ) const {
     return ( k < 0 || k >= size() ) ? Type() : root()->at( k );
                       // Need to go from 0, ..., n - 1
}

- The above implementation requires that `size()` returns in $\Theta(1)$ time
- We must have a member variable
```cpp
int tree_size;
```
    which stores the number of descendants of this node

- This requires $\Theta(n)$ additional memory
```cpp
template <typename Type>
bool BinarySearchTree<Type>::size() const {
    return root()->size();
}
```
- We can implement this in the `BinaryNode` class, if we want
    - The constructor will set the size to 1
    - We must now update `insert` and `erase` to update `tree_size`

In [32]:
{
    BinarySearchTree<int> t;
    for(int i : {42,39,11,8,3,29,19,16,24,34,70,51,47,59,54,52,68,63,99,92,81,75,87,96})
        t.insert(i);
    for(int i : {75,8,39,99,42,47})
        t.erase(i);
    std::cout << "At index 0: " << t.at(0) << std::endl;
    std::cout << "At index 6: " << t.at(6) << std::endl;
    std::cout << "At index 10: " << t.at(10) << std::endl;
    std::cout << "At index 14: " << t.at(14) << std::endl;
}

At index 0: 3
At index 6: 34
At index 10: 59
At index 14: 81


## Run Time: $O(h)$

- Almost all of the relevant operations on a binary search tree are $O(h)$
    - If the tree is close to a linked list, the run times is $O(n)$
        - Insert $1, 2, 3, 4, 5, 6, 7, \ldots, n$ into a empty binary search tree
    - The best we can do is if the tree is perfect, $O(\ln(n))$
    - Our goal will be to find tree structures where we can maintain a height of $\Theta(\ln(n))$

- We will look trees which ensure that the height remains $\Theta(\ln(n))$
    - AVL trees
    - B+ trees
    - Others exist, too

Based on material provided by Douglas Wilhelm Harder